In [1]:
import numpy as np
import pandas as pd

In [2]:
test = pd.read_csv("test_1zqHu22.csv")
train = pd.read_csv("train_0OECtn8.csv")

In [3]:
test['vid_cat'] = test['category_id'].astype(str) + test['video_id'].astype(str)
train['vid_cat'] = train['category_id'].astype(str) + train['video_id'].astype(str)
test['vid_cat'] = test['vid_cat'].astype('int64')
train['vid_cat'] = train['vid_cat'].astype('int64')

In [4]:
test.drop(["followers","views"],axis=1,inplace=True)
train.drop(["followers","views"],axis=1,inplace=True)

In [5]:
categorical_columns  = ["gender","profession" ]
train_encode = pd.get_dummies(data = train, prefix = 'OHE', prefix_sep='_',
               columns = categorical_columns,
               drop_first =True,
              dtype='int8')

In [6]:
test_encode = pd.get_dummies(data = test, prefix = 'OHE', prefix_sep='_',
               columns = categorical_columns,
               drop_first =True,
              dtype='int8')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_encode.drop('row_id',axis=1,inplace=True)
X = train_encode.drop('engagement_score',axis=1) # Independet variable
y = train_encode['engagement_score'] # dependent variable

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=17)

In [9]:
from xgboost import XGBRegressor

In [10]:
model = XGBRegressor()

In [11]:
model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [12]:
row_id = test_encode['row_id']
test_encode.drop('row_id',axis=1,inplace=True)
y_test_pred = model.predict(test_encode)
data = np.column_stack((row_id, y_test_pred))
sub = pd.DataFrame(data=data,columns=["row_id","engagement_score"])
sub["row_id"] = sub["row_id"].astype("int64")
sub.to_csv("submission_xgb_3.csv",index=False)